In [ ]:
#|hide
#|skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

# Tutorial - Distributed training in a notebook!
> Using `Accelerate` to launch a training script from your notebook

In [ ]:
#|all_multicuda

## Overview

In this tutorial we will see how to use [Accelerate](https://github.com/huggingface/accelerate) to launch a training function on a distributed system, from inside your **notebook**! 

To keep it easy, this example will follow training PETs, showcasing how all it takes is 3 new lines of code to be on your way!

## Setting up imports and building the DataLoaders

Along with the normal `from fastai.module.all import *` imports, two new ones need to be added:
```diff
+ from fastai.accelerate import *
from fastai.vision.all import *
from fastai.vision.models.xresnet import *

+ from accelerate import notebook_launcher
```

The first brings in a new `Learner.accelerate_ctx` context manager, similar to the `Learner.distrib_ctx`, however only it is capible to be ran from a Jupyter Notebook. The second brings in Accelerate's [notebook_launcher](https://huggingface.co/docs/accelerate/launcher), the key function we will call to run what we want.

In [ ]:
#hide
from fastai.vision.all import *
from fastai.accelerate import *
from fastai.vision.models.xresnet import *

from accelerate import notebook_launcher

Let's download some data to train on. You don't need to worry about using `rank0_first`, as since we're in our Jupyter Notebook it will only run on one process like normal:

In [ ]:
path = untar_data(URLs.PETS)

Next, we wrap the creation of the `DataLoaders`, our `vision_learner`, and call to `fine_tune` inside of a `train` function. 

> Note: It is important to **not** build the `DataLoaders` outside of the function, as absolutely *nothing* can be loaded onto CUDA beforehand.

In [ ]:
def get_y(o): return o[0].isupper()
def train(path):
    dls = ImageDataLoaders.from_name_func(
        path, get_image_files(path), valid_pct=0.2,
        label_func=get_y, item_tfms=Resize(224))
    learn = vision_learner(dls, resnet34, metrics=error_rate).to_fp16()
    learn.fine_tune(1)

The last addition to the `train` function needed is to use our context manager before calling `fine_tune` and setting `in_notebook` to `True`:

> Note: for this example `sync_bn` is disabled for compatibility purposes with `torchvision`'s resnet34

In [ ]:
def train(path):
    dls = ImageDataLoaders.from_name_func(
        path, get_image_files(path), valid_pct=0.2,
        label_func=get_y, item_tfms=Resize(224))
    learn = vision_learner(dls, resnet34, metrics=error_rate).to_fp16()
    with learn.accelerate_ctx(sync_bn=False, in_notebook=True):
        learn.fine_tune(1)
    if not rank_distrib():
        learn.export("pets")

`if not rank_distrib():` checks if you are on the main process or not, and in this case if you are you export your Learner only once. 

Finally, just call `notebook_launcher`, passing in the training function, any arguments as a tuple, and the number of GPUs (processes) to use:

In [ ]:
notebook_launcher(train, (path,), num_processes=2)

Launching training on 2 GPUs.
Training Learner...


epoch,train_loss,valid_loss,error_rate,time
0,0.342019,0.228441,0.105041,00:54


epoch,train_loss,valid_loss,error_rate,time
0,0.197188,0.141764,0.062246,00:56


Afterwards we can import our exported `Learner`, save, or anything else we may want to do in our Jupyter Notebook outside of a distributed process

In [ ]:
imgs = get_image_files(path)
learn = load_learner(path/'pets')
learn.predict(imgs[0])

('False', TensorBase(0), TensorBase([0.9718, 0.0282]))